In [10]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator

from indra_db_lite.api import get_entrez_pmids_for_hgnc
from indra_db_lite.api import get_entrez_pmids_for_uniprot
from indra_db_lite.api import get_plaintexts_for_text_ref_ids
from indra_db_lite.api import get_text_ref_ids_for_agent_text
from indra_db_lite.api import get_text_ref_ids_for_pmids


from adeft_indra.grounding import AdeftGrounder
from adeft_indra.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename

In [11]:
def get_text_ref_ids_for_entity(ns, id_):
    if ns == 'HGNC':
        pmids = get_entrez_pmids_for_hgnc(id_)
    elif ns == 'UP':
        pmids = get_entrez_pmids_for_uniprot(id_)
    return list(get_text_ref_ids_for_pmids(pmids).values())

In [12]:
adeft_grounder = AdeftGrounder()

In [13]:
shortforms = ['PCD']
model_name = '&'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../../', 'results', model_name))

In [14]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    text_ref_ids = get_text_ref_ids_for_agent_text(shortform)
    content = get_plaintexts_for_text_ref_ids(text_ref_ids, contains=shortforms)
    text_dict = content.flatten()
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [15]:
list(zip(longforms, counts))

[('programmed cell death', 1696),
 ('primary ciliary dyskinesia', 227),
 ('percutaneous catheter drainage', 22),
 ('paraneoplastic cerebellar degeneration', 22),
 ('primary carnitine deficiency', 12),
 ('percutaneous drainage', 9),
 ('passive cavitation detection', 8),
 ('premature centromere division', 6),
 ('plant cell death', 6),
 ('perfused capillary density', 5),
 ('photon counting detector', 5),
 ('photocatalytic degradation', 5),
 ('physiological cell death', 5),
 ('phlegmasia cerulea dolens', 4),
 ('post challenge day', 4),
 ('proteins c deficiency', 4),
 ('plasma cell dyscrasias', 3),
 ('plasma cell differentiation', 3),
 ('protocatechuate 3 4 dioxygenase', 3)]

In [16]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

In [17]:
result = [grounding_map, names, pos_labels]

In [18]:
result

[{'paraneoplastic cerebellar degeneration': 'NCIT:NCIT:C4685',
  'passive cavitation detection': 'ungrounded',
  'percutaneous catheter drainage': 'ungrounded',
  'percutaneous drainage': 'ungrounded',
  'perfused capillary density': 'ungrounded',
  'phlegmasia cerulea dolens': 'ungrounded',
  'photocatalytic degradation': 'ungrounded',
  'photon counting detector': 'ungrounded',
  'physiological cell death': 'GO:GO:0012501',
  'plant cell death': 'GO:GO:0008219',
  'plasma cell differentiation': 'GO:GO:0002317',
  'plasma cell dyscrasias': 'ungrounded',
  'post challenge day': 'ungrounded',
  'premature centromere division': 'ungrounded',
  'primary carnitine deficiency': 'MESH:C536778',
  'primary ciliary dyskinesia': 'MESH:D002925',
  'programmed cell death': 'GO:GO:0012501',
  'proteins c deficiency': 'ungrounded',
  'protocatechuate 3 4 dioxygenase': 'ungrounded'},
 {'NCIT:NCIT:C4685': 'Paraneoplastic Cerebellar Degeneration',
  'GO:GO:0012501': 'programmed cell death',
  'GO:GO:0

In [19]:
grounding_map, names, pos_labels = [{'paraneoplastic cerebellar degeneration': 'NCIT:NCIT:C4685',
  'passive cavitation detection': 'ungrounded',
  'percutaneous catheter drainage': 'ungrounded',
  'percutaneous drainage': 'ungrounded',
  'perfused capillary density': 'ungrounded',
  'phlegmasia cerulea dolens': 'ungrounded',
  'photocatalytic degradation': 'ungrounded',
  'photon counting detector': 'ungrounded',
  'physiological cell death': 'GO:GO:0012501',
  'plant cell death': 'GO:GO:0008219',
  'plasma cell differentiation': 'GO:GO:0002317',
  'plasma cell dyscrasias': 'ungrounded',
  'post challenge day': 'ungrounded',
  'premature centromere division': 'ungrounded',
  'primary carnitine deficiency': 'MESH:C536778',
  'primary ciliary dyskinesia': 'MESH:D002925',
  'programmed cell death': 'GO:GO:0012501',
  'proteins c deficiency': 'ungrounded',
  'protocatechuate 3 4 dioxygenase': 'ungrounded'},
 {'NCIT:NCIT:C4685': 'Paraneoplastic Cerebellar Degeneration',
  'GO:GO:0012501': 'programmed cell death',
  'GO:GO:0008219': 'cell death',
  'GO:GO:0002317': 'plasma cell differentiation',
  'MESH:C536778': 'Systemic carnitine deficiency',
  'MESH:D002925': 'Ciliary Motility Disorders'},
 ['GO:GO:0012501', 'MESH:D002925', 'NCIT:NCIT:C4685']]

In [20]:
excluded_longforms = []

In [21]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [22]:
additional_entities = {}

In [23]:
unambiguous_agent_texts = {}

In [24]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts(
    (text, text_ref_id) for text_ref_id, text in all_texts.items()
)
agent_text_text_ref_id_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_text_ref_id_map[label].append(id_)

entity_text_ref_id_map = {
    entity: set(
        get_text_ref_ids_for_entity(*entity.split(':', maxsplit=1))
    )
    for entity in additional_entities
}

In [25]:
intersection1 = []
for entity1, trids1 in entity_text_ref_id_map.items():
    for entity2, trids2 in entity_text_ref_id_map.items():
        intersection1.append((entity1, entity2, len(trids1 & trids2)))

In [26]:
intersection2 = []
for entity1, trids1 in agent_text_text_ref_id_map.items():
    for entity2, pmids2 in entity_text_ref_id_map.items():
        intersection2.append((entity1, entity2, len(set(trids1) & trids2)))

In [27]:
intersection1

[]

In [28]:
intersection2

[]

In [29]:
all_used_trids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_trids = set()
    for agent_text in agent_texts[1]:
        trids = set(get_text_ref_ids_for_agent_text(agent_text))
        new_trids = list(trids - all_texts.keys() - used_trids)
        content = get_plaintexts_for_text_ref_ids(new_trids, contains=agent_texts[1])
        text_dict = content.flatten()
        corpus.extend(
            [
                (text, entity, trid) for trid, text in text_dict.items() if len(text) >= 5
            ]
        )
        used_trids.update(new_trids)
    all_used_trids.update(used_trids)
        
for entity, trids in entity_text_ref_id_map.items():
    new_trids = list(set(trids) - all_texts.keys() - all_used_trids)
    _, contains = additional_entities[entity]
    content = get_plaintexts_for_text_ref_ids(new_trids, contains=contains)
    text_dict = content.flatten()
    corpus.extend(
        [
            (text, entity, trid) for trid, text in text_dict.items() if len(text) >= 5
        ]
    )

In [30]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [31]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2021-10-09 16:06:51] /adeft/Py/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2021-10-09 16:07:17] /adeft/Py/adeft/adeft/modeling/classify.py - Best f1 score of 0.9885517189307473 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [32]:
classifier.stats

{'label_distribution': {'GO:GO:0012501': 1283,
  'MESH:D002925': 136,
  'ungrounded': 48,
  'MESH:C536778': 10,
  'NCIT:NCIT:C4685': 12,
  'GO:GO:0008219': 5,
  'GO:GO:0002317': 2},
 'f1': {'mean': 0.988552, 'std': 0.003552},
 'precision': {'mean': 0.982087, 'std': 0.00451},
 'recall': {'mean': 0.99511, 'std': 0.003558},
 'GO:GO:0002317': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'GO:GO:0008219': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'GO:GO:0012501': {'f1': {'mean': 0.989948, 'std': 0.002243},
  'pr': {'mean': 0.997662, 'std': 0.001909},
  'rc': {'mean': 0.982361, 'std': 0.003861}},
 'MESH:C536778': {'f1': {'mean': 0.8, 'std': 0.4},
  'pr': {'mean': 0.8, 'std': 0.4},
  'rc': {'mean': 0.8, 'std': 0.4}},
 'MESH:D002925': {'f1': {'mean': 0.974126, 'std': 0.019073},
  'pr': {'mean': 0.970635, 'std': 0.02765},
  'rc': {'mean': 0.978022, 'std': 0.017973}},
 'NCI

In [33]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [34]:
disamb.dump(model_name, results_path)

In [35]:
print(disamb.info())

Disambiguation model for PCD

Produces the disambiguations:
	Ciliary Motility Disorders*	MESH:D002925
	Paraneoplastic Cerebellar Degeneration*	NCIT:NCIT:C4685
	Systemic carnitine deficiency	MESH:C536778
	cell death	GO:GO:0008219
	plasma cell differentiation	GO:GO:0002317
	programmed cell death*	GO:GO:0012501

Class level metrics:
--------------------
Grounding                             	Count	F1     
                 programmed cell death*	1283	0.98995
            Ciliary Motility Disorders*	 136	0.97413
                            Ungrounded	  48	0.79742
Paraneoplastic Cerebellar Degeneration*	  12	    1.0
         Systemic carnitine deficiency	  10	    0.8
                            cell death	   5	    0.0
           plasma cell differentiation	   2	    0.0

Global Metrics:
-----------------
	F1 score:	0.98855
	Precision:	0.98209
	Recall:		0.99511

* Positive labels
See Docstring for explanation



In [36]:
model_to_s3(disamb)

In [26]:
from adeft.disambiguate import load_disambiguator

In [27]:
disamb = load_disambiguator("BAL")

In [28]:
disamb

In [29]:
print(_28.info())

Disambiguation model for BAL

Produces the disambiguations:
	Bronchoalveolar Lavage	MESH:D018893
	CEL*	HGNC:1848
	Liver, Artificial	MESH:D019164
	benzaldehyde lyase*	MESH:C059416
	betaine aldehyde*	CHEBI:CHEBI:15710
	dimercaprol*	CHEBI:CHEBI:64198

Class level metrics:
--------------------
Grounding             	Count	F1     
Bronchoalveolar Lavage	1259	 0.9929
                   CEL*	  36	    1.0
     Liver, Artificial	  18	0.83619
            Ungrounded	  17	   0.65
           dimercaprol*	   8	    0.4
    benzaldehyde lyase*	   3	    0.2
      betaine aldehyde*	   2	    0.2

Global Metrics:
-----------------
	F1 score:	0.90773
	Precision:	1.0
	Recall:		0.83293

* Positive labels
See Docstring for explanation



/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is 